# Indeterminate Beam calculator

### Table of Contents
1. Initialisation
2. Basic usage (Readme example)
    - 2(b). Additional features (Readme example)
3. Support class breakdown
4. Load classes breakdown
5. Statically determinate beam (Ex 12.14 Hibbeler)
6. Statically Indeterminate beam (Ex 12.21 Hibbeler)
7. Spring supported beam deflection (Ex 12.16 Hibbeler)
8. Axial Loaded Indeterminate Beam (Ex 4.13 Hibbeler)

Note:
You must always run the initialization cell first. 
Then you are free to move to any of the other sections as per the table of contents.

 ## 1. INITIALIZATION


In [ ]:
# RUN THIS CELL FIRST TO INITIALISE GOOGLE NOTEBOOK!!!!
!pip install indeterminatebeam
%matplotlib inline

# import beam and supports
from indeterminatebeam import Beam, Support

# import loads (all load types imported for reference)
from indeterminatebeam import (
    PointTorque,
    PointLoad,
    PointLoadV,
    PointLoadH,
    UDL,
    UDLV,
    UDLH,
    TrapezoidalLoad,
    TrapezoidalLoadV,
    TrapezoidalLoadH,
    DistributedLoad,
    DistributedLoadV,
    DistributedLoadH
)

# Note: load ending in V are vertical loads
# load ending in H are horizontal loads
# load not ending in either takes angle as an input (except torque)


In [1]:
# NOT FOR USE ONLINE.
# Use this instead for initialization if running directly from package folder,
import sys, os
sys.path.insert(0, os.path.abspath('../../'))

# import beam and supports
from indeterminatebeam import Beam, Support

# import loads (all load types imported for reference)
from indeterminatebeam.loading import (
    PointTorque,
    PointLoad,
    PointLoadV,
    PointLoadH,
    UDL,
    UDLV,
    UDLH,
    TrapezoidalLoad,
    TrapezoidalLoadV,
    TrapezoidalLoadH,
    DistributedLoad,
    DistributedLoadV,
    DistributedLoadH
)

# Note: load ending in V are vertical loads
# load ending in H are horizontal loads
# load not ending in either takes angle as an input (except torque)


C:\Users\Jesse\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Jesse\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Jesse\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


## 2. Basic Usage (Readme example)

In [5]:
##arbritrary example defined in README.md
beam = Beam(7000)                          # Initialize a Beam object of length 9000 mm with E and I as defaults
beam_2 = Beam(9000,E=2000, I =100000)      # Initializa a Beam specifying some beam parameters

a = Support(5000,(1,1,0))                  # Defines a pin support at location x = 5000 mm  
b = Support(0,(0,1,0))                  # Defines a roller support at location x = 0 mm
c = Support(7000,(1,1,1))                  # Defines a fixed support at location x = 7000 mm
beam.add_supports(a,b,c)    

load_1 = PointLoadV(1000,2000)                # Defines a point load of 1000 N acting up, at location x = 2000 mm
load_2 = DistributedLoadV(2,(1000,4000))      # Defines a 2 N/mm UDL from location x = 1000 mm to x = 4000 mm 
load_3 = PointTorque(2*10**6, 3500)            # Defines a 2*10**6 N.mm point torque at location x = 3500 mm
beam.add_loads(load_1,load_2,load_3)    # Assign the support objects to a beam object created earlier

beam.analyse()


fig_1 = beam.plot_beam_external()
fig_1.show()

fig_2 = beam.plot_beam_internal()
fig_2.show()

fig_1.update_layout(width =600)
fig_2.update_layout(width =600)

#fig_1.write_image("./readme_example_external.png")
#fig_2.write_image("./readme_example_internal.png")

fig_1.write_html("./readme_example_external.html")
fig_2.write_html("./readme_example_internal.html")

## 2(b). Additional Features (Readme example)

In [ ]:
# Run section 2 (prior to running this example)   

##query for the data at a specfic point
print("bending moments at 3000 mm: " + str(beam.get_bending_moment(3000)))
print("shear forces at 1,2,3,4,5m points: " + str(beam.get_shear_force(1000,2000,3000,4000,5000)))
print("normal force absolute max: " + str(beam.get_normal_force(return_absmax=True)))
print("deflection max: " + str(beam.get_deflection(return_max = True)))   

##add a query point to a plot (adds values on plot)
beam.add_query_points(1000,3000,5000)
beam.remove_query_points(5000)

## plot the results for the beam
fig = beam.plot_beam_internal()
fig.show()

## 3. Support class breakdown

In [ ]:
# The parameters for a support class are as below, taken from the docstring
# for the Support class __init__ method.

# Parameters:
#         -----------
#         coord: float
#             x coordinate of support on a beam (default 0)
#         fixed: tuple of 3 booleans
#             Degrees of freedom that are fixed on a beam for movement in
#             x, y and bending, 1 represents fixed and 0 represents free
#             (default (1,1,1))
#         kx :
#             stiffness of x support (kN/mm), if set will overide the
#             value placed in the fixed tuple. (default = None)
#         ky : (positive number)
#             stiffness of y support (kN/mm), if set will overide the
#             value placed in the fixed tuple. (default = None)


# Lets define every possible degree of freedom combination for
# supports below, and view them on a plot:
support_0 = Support(0, (1,1,1))     # conventional fixed support
support_1 = Support(1000, (1,1,0))     # conventional pin support
support_2 = Support(2000, (1,0,1))     
support_3 = Support(3000, (0,1,1))
support_4 = Support(4000, (0,0,1))
support_5 = Support(5000, (0,1,0))     # conventional roller support
support_6 = Support(6000, (1,0,0))

# Note we could also explicitly define parameters as follows:
support_0 = Support(coord=0, fixed=(1,1,1))

# Now lets define some spring supports
support_7 = Support(7000, (0,0,0), kx = 10000)    #spring in x direction only
support_8 = Support(8000, (0,0,0), ky = 5000)     # spring in y direction only
support_9 = Support(9000, (0,0,0), kx = 100000, ky = 100000)     # spring in x and y direction

# Now lets define a support which is fixed in one degree of freedom
# but has a spring stiffness in another degree of freedom
support_10 = Support(10000, (0,1,0), kx = 10) #spring in x direction, fixed in y direction
support_11 = Support(11000, (0,1,1), kx = 10) #spring in x direction, fixed in y and m direction

# Note we could also do the following for the same result since the spring
# stiffness overides the fixed boolean in respective directions
support_10 = Support(10000, (1,1,0), kx =10)

# Now lets plot all the supports we have created
beam = Beam(11000)

beam.add_supports(
    support_0,
    support_1,
    support_2,
    support_3,
    support_4,
    support_5,
    support_6,
    support_7,
    support_8,
    support_9,
    support_10,
    support_11,
)

fig = beam.plot_beam_diagram()
fig.show()

## 4. Load classes breakdown

### 4.1 Point Torque

In [ ]:
# defined using a force (technically a moment, however force is used to maintain consistenct for all load classes) and a coordinate. An anti-clockwise moment is positive by convention of this package.
load_1 = PointTorque(force=1000000, coord=1000)
load_2 = PointTorque(force=-1000000, coord=2000)

# Plotting the loads
beam = Beam(3000)
beam.add_loads(
    load_1,
    load_2
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.2 Point Load

In [ ]:
# defined by force, coord and angle (0)
load_1 = PointLoad(force=1000, coord=1000, angle=0)
load_2 = PointLoad(force=1000, coord=2000, angle=45)
load_3 = PointLoad(force=1000, coord=3000, angle=90)

# Plotting the loads
beam = Beam(4000)
beam.add_loads(
    load_1,
    load_2,
    load_3
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.3 Uniformly Distributed Load (UDL)

In [ ]:
# defined by force, span (tuple with start and end point) 
# and angle of force
load_1 = UDL(force=1, span=(1000,2000), angle = 0)
load_2 = UDL(force=1, span=(3000,4000), angle = 45)
load_3 = UDL(force=1, span=(5000,6000), angle = 90)

# Plotting the loads
beam = Beam(7000)
beam.add_loads(
    load_1,
    load_2,
    load_3
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.4 Trapezoidal Load

In [ ]:
# defined by force (tuple with start and end force), 
# span (tuple with start and end point) and angle of force
load_1 = TrapezoidalLoad(force=(1,2), span=(1000,2000), angle = 0)
load_2 = TrapezoidalLoad(force=(-1,-2), span=(3000,4000), angle = 45)
load_3 = TrapezoidalLoad(force=(-1,2), span=(5000,6000), angle = 90)

# Plotting the loads
beam = Beam(7000)
beam.add_loads(
    load_1,
    load_2,
    load_3
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.5 Distributed Load

In [ ]:
# defined with Sympy expression of the distributed load function 
# expressed using variable x which represents the beam x-coordinate. 
# Requires quotation marks around expression. As with the UDL and 
# Trapezoidal load classes other parameters to express are the span 
# (tuple with start and end point) and angle of force.
# NOTE: where UDL or Trapezoidal load classes can be spefied (linear functions)
# they should be used for quicker analysis times.

load_1 = DistributedLoad(expr= "2", span=(1000,2000), angle = 0)
load_2 = DistributedLoad(expr= "2*(x/1000-6)**2 -5", span=(3000,4000), angle = 45)
load_3 = DistributedLoad(expr= "cos(0.005*x)", span=(5000,6000), angle = 90)

# Plotting the loads
beam = Beam(7000)
beam.add_loads(
    load_1,
    load_2,
    load_3
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.6 Vertical and Horizontal load child classes

In [ ]:
# for all loads except the point torque an angle is specified for the
# direction of the load. If the load to be specified is to be completely
# vertical or completely horizontal a V (vertical) or a H (horizontal)
# can be added at the end of the class name, and the angle does then
# not need to be spefied.

# The following two loads are equivalent horizontal loads
load_1 = PointLoad(force=1000, coord=1000, angle = 0)
load_2 = PointLoadH(force=1000, coord=2000)

# The following two loads are equivalent vertical loads
load_3 = PointLoad(force=1000, coord=3000, angle = 90)
load_4 = PointLoadV(force=1000, coord=4000)

# The following two loads are also equivalent (a negative sign
# esentially changes the load direction by 180 degrees).
load_5 = PointLoad(force=1000, coord=5000, angle = 0)
load_6 = PointLoad(force=-1000, coord=6000, angle = 180)


# Plotting the loads
beam = Beam(7000)
beam.add_loads(
    load_1,
    load_2,
    load_3,
    load_4,
    load_5,
    load_6
)

fig = beam.plot_beam_diagram()
fig.show()

## 5. STATICALLY DETERMINATE BEAM (Ex 12.14 Hibbeler)

In [ ]:
## Statically Determinate beam (Ex 12.14 Hibbeler)
## Determine the reactions at the roller support B of the beam described below: 
## 3m long, fixed at A (x = 0m), roller support at B (x=3m), vertical point load at midpan of 8kN, UDL of 6kN/m, EI constant.

beam = Beam(8000, E=1, I = 1)     ##EI Defined to be 1 get the deflection as a function of EI

a = Support(0, (1,1,1))             ##explicitly stated although this is equivalent to Support() as the defaults are for a cantilever on the left of the beam.

load_1 = TrapezoidalLoadV((-4,0),(0,6000))

beam.add_supports(a)
beam.add_loads(load_1)

beam.analyse()
print(f"Deflection is {beam.get_deflection(8000)} N.mm3 / EI ")

fig = beam.plot_beam_internal()  
fig.show()
# Note: all plots are correct, deflection graph shape is correct but for actual deflection values will need real EI properties.

##save the results as a pdf (optional)
# Can save figure using `fig.write_image("./results.pdf")` (can change extension to be
# png, jpg, svg or other formats as reired). Requires pip install -U kaleido


## 6. STATICALLY INDETERMINATE BEAM (Ex 12.21 Hibbeler)

In [ ]:
# Statically Indeterminate beam (Ex 12.21 Hibbeler)
# Determine the reactions at the roller support B of the beam described below: 
# 3000 mm long, fixed at A (x = 0 mm), roller support at B (x=3000 mm), 
# vertical point load at midpan of 8 kN, UDL of 6 N/mm, EI constant.

beam = Beam(3000)

a = Support(0,(1,1,1))  
b = Support(3000,(0,1,0))

load_1 = PointLoadV(-8000,1500)
load_2 = UDLV(-6, (0,3000))

beam.add_supports(a,b)
beam.add_loads(load_1,load_2)

beam.analyse()

print(f"The beam has an absolute maximum shear force of: {beam.get_shear_force(return_absmax=True)} N")
print(f"The beam has an absolute maximum bending moment of: {beam.get_bending_moment(return_absmax=True)} N.mm")
print(f"The beam has a vertical reaction at B of: {beam.get_reaction(3000,'y')} N")

fig1 = beam.plot_beam_external()  
fig1.show()

fig2 = beam.plot_beam_internal()  
fig2.show()

# fig1.write_image("./example_1_external.png")
# fig2.write_image("./example_1_internal.png")




## 7. SPRING SUPPORTED BEAM (Ex 12.16 Hibbeler)

In [ ]:
## Spring Supported beam (Ex 12.16 Hibbeler)
## Determine the vertical displacement at x = 1000 mm for the beam detailed below:
## 3000 mm long, spring of ky = 45000 N/mm at A (x = 0 mm) and B (x = 3000 mm), vertical point load at x = 1000 mm of 3000 N, E = 200 GPa, I = 4.6875*10**-6 m4.

##when initializing beam we should specify E and I. Units should be expressed in MPa (N/mm2) for E, and mm4 for I
beam = Beam(3000, E=(200)*10**3, I=(4.6875*10**-6)*10**12)

##creating supports, note that an x support must be specified even when there are no x forces. This will not affect the accuracy or reliability of results.
##Also note that ky units are kN/m in the problem but must be in kN/mm for the program to work correctly.
a = Support(0, (1,1,0), ky = 45000)   
b = Support(3000, (0,1,0), ky = 45000)

load_1 = PointLoadV(-3000,1000)

beam.add_supports(a,b)
beam.add_loads(load_1)

beam.analyse()

beam.get_deflection(1000)

fig1 = beam.plot_beam_external()  
fig1.show()

fig2 = beam.plot_beam_internal()  
fig2.show()

##results in 38.46 mm deflection ~= 38.4mm specified in textbook (difference only due to their rounding)
##can easily check reliability of answer by looking at deflection at the spring supports. Should equal F/k.
## ie at support A (x = 0 m), the reaction force is 2kN by equilibrium, so our deflection is F/K = 2kn / 45*10-3 kN/mm = 44.4 mm (can be seen in plot)

## 8. AXIAL LOADED INDETERMINATE BEAM (Ex 4.13 Hibbeler)

In [ ]:
##AXIAL LOADED INDETERMINATE BEAM (Ex 4.13 Hibbeler)
## A rod with constant EA has a force of 60kN applied at x =0.1, and the beam has fixed supports at x=0, and x =0.4. Determine the reaction forces.

beam = Beam(400)

a = Support()
b = Support(400)

load_1 = PointLoadH(-60000, 100)

beam.add_supports(a,b)
beam.add_loads(load_1)

beam.analyse()
beam.plot_normal_force()
